In [1]:
!nvidia-smi

Sat Jan 22 17:51:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100S-PCI...  Off  | 00000000:00:07.0 Off |                    0 |
| N/A   34C    P0    52W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import wandb

%env WANDB_ENTITY = arampacha
wandb_entity = os.environ["WANDB_ENTITY"]

%env WANDB_PROJECT = xlsr-hy
wandb_project = os.environ["WANDB_PROJECT"]

%env WANDB_LOG_MODEL = false
%env WANDB_WATCH = false

env: WANDB_ENTITY=arampacha
env: WANDB_PROJECT=xlsr-hy
env: WANDB_LOG_MODEL=false
env: WANDB_WATCH=false


In [3]:
%env HF_HOME = /workspace/cache/hf/

env: HF_HOME=/workspace/cache/hf/


In [4]:
import numpy as np

In [5]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_7_0", "hy-AM", split="train+validation", use_auth_token=True)
common_voice_test = load_dataset("mozilla-foundation/common_voice_7_0", "hy-AM", split="test", use_auth_token=True)

Reusing dataset common_voice (/workspace/cache/hf/datasets/mozilla-foundation___common_voice/hy-AM/7.0.0/33e08856cfa0d0665e837bcad73ffd920a0bc713ce8c5fffb55dbdf1c084d5ba)
Reusing dataset common_voice (/workspace/cache/hf/datasets/mozilla-foundation___common_voice/hy-AM/7.0.0/33e08856cfa0d0665e837bcad73ffd920a0bc713ce8c5fffb55dbdf1c084d5ba)


In [6]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "locale", "segment", "up_votes"])

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence,gender
0,Իրբենե մուտքը հնարավոր էր միայն հատուկ թույտվութուն ունենալու դեպքում։,female
1,Կարմիր քացախից պատրաստված սոուսի հետ համահունչ են լոլիկը և քաղցր պղպեղը։,
2,Քննադատները շեշտել են ռոքի էլեկտրականությունն ու ոճական բազմազանությունը։,female
3,"Եվ այսպես, այն շարունակվեց մի քանի տարի։",
4,Փոքր հասակից խաղացել է հարձակման գծում։,female
5,Ֆոքստրոտի զարգմացման մեջ մեծ ավանդ են ունեցել այդ ժամանակների հայտնի պարողները։,female
6,Նա հիվանդ էր նաև սիֆիլիսով։,male
7,Թանգարանի զարդարանքներու մեծ մասն ընտրուած է մրցոյթներու միջոցով։,male
8,Տաքարյուն կենդանիների օրգանիզմի մակարդակում պահպանվում է մարմնի հաստատուն ջերմաստիճան։,male
9,Երկաթյա դիվիզիայի բոլոր զինվորները կրում էին գանգերի խորհրդանշաններով գլխարկներ։,male


In [9]:
def extract_all_chars(batch):
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [10]:
import re
chars_to_remove_regex = re.compile('[\,\?\.\!\-\;\:\"\“\%\‘\”\�\'«»\(\)։՝՞՛]')

def normalize_text(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()+" "
    return batch

In [11]:
common_voice_train = common_voice_train.map(normalize_text)
common_voice_test = common_voice_test.map(normalize_text)

Loading cached processed dataset at /workspace/cache/hf/datasets/mozilla-foundation___common_voice/hy-AM/7.0.0/33e08856cfa0d0665e837bcad73ffd920a0bc713ce8c5fffb55dbdf1c084d5ba/cache-92b6fb1731c9c846.arrow
Loading cached processed dataset at /workspace/cache/hf/datasets/mozilla-foundation___common_voice/hy-AM/7.0.0/33e08856cfa0d0665e837bcad73ffd920a0bc713ce8c5fffb55dbdf1c084d5ba/cache-6eaf4a994dc115a8.arrow


In [12]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
all_chars_train = sorted(vocab_train["vocab"][0])
all_chars_test = sorted(vocab_test["vocab"][0])
assert all_chars_train == all_chars_test
print("".join(all_chars_train))
print("".join(all_chars_test))

 աբգդեզէըթժիլխծկհձղճմյնշոչպջռսվտրցւփքօֆև
 աբգդեզէըթժիլխծկհձղճմյնշոչպջռսվտրցւփքօֆև


Let's look at the processed text labels again.

In [14]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence,gender
0,այդ ամենը խանգարում էր բրիտանական ուժերին ընդհատել քաղաքի պաշարումը,female
1,աճում է բրոնզե իրերի քանակը,female
2,խնդրի լուծման շարունակությունը կախված է մարմնի շարժման ուղեծրի տեսակից,male
3,հ բաչկովի մարզական կարիերան հարթ չի ընթացել,male
4,ոսկրային հյուսվածքի թերզարգացումը կարող է լինել մասնակի կամ լրիվ,
5,առևտուրը երկրի ներսում կոչվում է ներքին այլ երկրների հետ կատարվողը արտաքին,male
6,հաճախ պատկերվում է առատության եղջյուրով,
7,նա նայելով բացականչել է հիանալի է հիանալի է,
8,էգերը մոխրագույն են,male
9,առանձնահատուկ կարևոր տեղ է զբաղեցնում ժամանակակից չինական դրամատուրգիայում,male


In [15]:
repo_name = "wav2vec2-xls-r-300m-hy"

In [16]:
from transformers import (
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer
)

In [17]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(f"arampacha/{repo_name}", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [18]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [20]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [21]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

  0%|          | 0/554 [00:00<?, ?ex/s]

Loading cached processed dataset at /workspace/cache/hf/datasets/mozilla-foundation___common_voice/hy-AM/7.0.0/33e08856cfa0d0665e837bcad73ffd920a0bc713ce8c5fffb55dbdf1c084d5ba/cache-7b5e2079b66800b3.arrow


In [22]:
durations = common_voice_train.map(lambda x: {"duration": x/processor.feature_extractor.sampling_rate}, input_columns=["input_length"], keep_in_memory=True)

  0%|          | 0/554 [00:00<?, ?ex/s]

In [23]:
dur = np.array((durations["duration"]))
dur.min(), dur.max(), dur.mean(), dur.std()

(2.52, 10.404, 6.086794223826715, 1.7343751954715556)

In [24]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [25]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [26]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

In [27]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer, "cer":cer}

In [28]:
from torch.optim.lr_scheduler import LambdaLR

def get_tri_stage_schedule(
    optimizer, num_training_steps, ratios=[0.1, 0.4, 0.5], num_warmup_steps=None, num_hold_steps=None, start_ratio=0.01, end_ratio=0.05 
):
    assert (num_warmup_steps is None) == (num_hold_steps is None)
    if num_warmup_steps is None:
        num_warmup_steps = int(ratios[0]*num_training_steps)
        num_hold_steps = int(ratios[1]*num_training_steps)
        start_decay_step = num_warmup_steps + num_hold_steps
        a_w, b_w = (1-start_ratio)/num_warmup_steps, start_ratio
        num_decay_steps = num_training_steps - start_decay_step
        a_d, b_d = (end_ratio-1)/num_decay_steps, 1.
        
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return a_w * float(current_step) + b_w
        if current_step < start_decay_step:
            return 1.
        return max(end_ratio, a_d * float(current_step - start_decay_step) + b_d )
    
    return LambdaLR(optimizer, lr_lambda)

In [29]:
sweep_config = {
  "name": "hy-cv-simple",
  "method": "grid",
  "parameters" : {
    "bs" :{
      "values" : [64, 128, 256]
    },
    "lr" :{
      "values" : [2e-5, 5e-5, 1e-4, 3e-4]
    },
    "mask_time_prob" :{
      "values" : [0.3, 0.5, 0.7]
    },
    "hidden_dropout" :{
      "values" : [0., 0.1, 0.2]
    },
    "mask_feature_prob" :{
      "values" : [0.15, 0.25, 0.35]
    },
  }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: 0rmfn30y
Sweep URL: https://wandb.ai/arampacha/xlsr-hy/sweeps/0rmfn30y


In [30]:
def get_trainer(cfg):
    
    model = Wav2Vec2ForCTC.from_pretrained(
        "facebook/wav2vec2-xls-r-300m", 
        attention_dropout=0.0,
        hidden_dropout=cfg.hidden_dropout,
        feat_proj_dropout=0.0,
        mask_time_prob=cfg.mask_time_prob,
        mask_feature_prob=cfg.mask_feature_prob,
        mask_feature_length=64,
        layerdrop=0.1,
        ctc_loss_reduction="mean", 
        pad_token_id=processor.tokenizer.pad_token_id,
        vocab_size=len(processor.tokenizer),
    )
    model.freeze_feature_encoder()
    
    grad_accum = cfg.bs//32
    num_training_steps = 6400//grad_accum

    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.98), eps=1e-8, weight_decay=0.)
    scheduler = get_tri_stage_schedule(optimizer, num_training_steps)
    
    training_args = TrainingArguments(
        output_dir=repo_name,
        group_by_length=True,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=64,
        gradient_accumulation_steps=grad_accum,
        dataloader_num_workers=8,
        evaluation_strategy="steps",
        max_steps=num_training_steps,
        gradient_checkpointing=True,
        fp16=True,
        save_steps=6400//grad_accum,
        eval_steps=800//grad_accum,
        logging_steps=400//grad_accum,
        learning_rate=cfg.lr,
        # warmup_steps=500,
        save_total_limit=2,
        push_to_hub=False,
        run_name="xlsr-hy-test",
        report_to="all",
        label_smoothing_factor=0.1,
    )
    trainer = Trainer(
        model=model,
        data_collator=data_collator,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=common_voice_train,
        eval_dataset=common_voice_test,
        tokenizer=processor.feature_extractor,
        optimizers=(optimizer, scheduler)
    )
    return trainer

In [31]:
def train():
    with wandb.init() as run:
        cfg = wandb.config
        trainer = get_trainer(cfg)
        trainer.train()

In [ ]:
wandb.agent(sweep_id, function=train)